In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from keras.layers import Dense
from keras.models import Model
from keras.applications.resnet50 import preprocess_input, ResNet50
from keras.preprocessing.image import ImageDataGenerator

/home/jorisvanvugt/miniconda3/envs/dsb/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
base_model = ResNet50(include_top=False, input_shape=(256, 256, 3), pooling='avg')
for layer in base_model.layers:
    layer.trainable = False

In [3]:
x = Dense(512, activation='relu')(base_model.output)
x = Dense(2, activation='softmax')(x)
model = Model(inputs=base_model.inputs, outputs=x)
model.compile('adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 262, 262, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 128, 128, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 128, 128, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [4]:
batch_size = 32
train_gen = ImageDataGenerator(preprocessing_function=preprocess_input, zoom_range=0.1, width_shift_range=10,
                               rotation_range=10, vertical_flip=True, horizontal_flip=True, height_shift_range=10,
                               fill_mode='reflect').flow_from_directory('satdata/train', batch_size=batch_size)

val_gen = ImageDataGenerator(preprocessing_function=preprocess_input).flow_from_directory('satdata/val', shuffle=False, batch_size=batch_size)

Found 5553 images belonging to 2 classes.
Found 207 images belonging to 2 classes.


In [ ]:
model.fit_generator(train_gen, train_gen.n // batch_size, epochs=5, verbose=1,
                    validation_data=train_gen, validation_steps=val_gen.n // batch_size)

Epoch 1/5
173/173 [==============================] - 101s 581ms/step - loss: 0.3869 - acc: 0.8456 - val_loss: 0.3505 - val_acc: 0.8646
Epoch 2/5
173/173 [==============================] - 99s 574ms/step - loss: 0.2879 - acc: 0.8750 - val_loss: 0.5729 - val_acc: 0.8229
Epoch 3/5
173/173 [==============================] - 99s 574ms/step - loss: 0.2766 - acc: 0.8850 - val_loss: 0.5814 - val_acc: 0.8177
Epoch 4/5
145/173 [========================>.....] - ETA: 15s - loss: 0.2698 - acc: 0.8862